In [88]:
dias_semana = [(   'Sunday',       'Domingo'),
               (   'Monday', 'Segunda-feira'),
               (  'Tuesday',   'Terça-feira'),
               ('Wednesday',  'Quarta-feira'),
               ( 'Thursday',  'Quinta-feira'),
               (   'Friday',   'Sexta-feira'),
               ( 'Saturday',        'Sábado')]

In [89]:
import warnings
warnings.filterwarnings('ignore')

In [90]:
import pandas as pd
from code_class import GeoVictoria, FolhaPonto
from tqdm import tqdm
import datetime as dt

pd.set_option('display.max_rows', 100)

In [91]:
lista_tp_permis = []

for permission in GeoVictoria.GetPermissionList():
    lista_tp_permis.append((permission['ID_TIPO_PERMISO'], permission['DESCRIPCION_TIPO_PERMISO']))

In [92]:
df_funcionários = pd.DataFrame(GeoVictoria.GetUserList())

df_funcionários['Setor'] = df_funcionários['GroupDescription'].apply(lambda x:  '' if 'LOGIN' in x.upper() else
                                                                                'Garantia' if 'GARAN' in x.upper() else
                                                                                'Logistica' if 'LOGIS' in x.upper() else
                                                                                'Logistica' if 'LOGÍS' in x.upper() else
                                                                                'Vendas' if 'VENDA' in x.upper() else
                                                                                'Compras' if 'COMPR' in x.upper() else
                                                                                'Gente e Gestão' if 'GENT' in x.upper() else
                                                                                'Gente e Gestão' if 'DP' in x.upper() else
                                                                                'Estoque' if 'ESTOQ' in x.upper() else
                                                                                'Expedição' if 'EXPED' in x.upper() else
                                                                                'Almoxarifado' if 'ALMOX' in x.upper() else
                                                                                'Financeiro' if 'FINAN' in x.upper() else
                                                                                'Servicos Gerais' if 'GERAIS' in x.upper() else
                                                                                'E-commerce' if 'COMMERCE' in x.upper() else
                                                                                'Inteligencia Comercial' if 'COMERCIAL' in x.upper() else
                                                                                'Cobranca' if 'COBRAN' in x.upper() else
                                                                                'Marketing' if 'MARKET' in x.upper() else
                                                                                'Dados' if 'DADOS' in x.upper() else
                                                                                'Administrativo' if 'ADM' in x.upper() else
                                                                                'Supervisor' if 'CHEFE' in x.upper() else
                                                                                'Tecnologia' if 'TECNO' in x.upper() else
                                                                                'Projetos' if 'PROJE' in x.upper() else
                                                                                'Expansão' if 'EXPAN' in x.upper() else
                                                                                'Motorista' if 'MOTOR' in x.upper() else '')

df_funcionários['Nome completo'] = df_funcionários['Name'] + ' ' + df_funcionários['LastName']

for i in df_funcionários.index:
    nome = []
    for x in df_funcionários.loc[i, 'Nome completo'].split():
        nome_completo = ' '.join(nome)
        df_funcionários.loc[i, 'Nome completo'] = nome_completo

cod_fun = list(df_funcionários[df_funcionários['Enabled'] == 1]['Identifier'])

In [93]:
df_permis = pd.DataFrame(GeoVictoria.GetPermissions(person=','.join(cod_fun)))

In [94]:
contador = 1

df_horas = pd.DataFrame(GeoVictoria.GetAttendance(person=','.join(cod_fun[:48])))

while contador < int(len(cod_fun) / 48) + 1:
    df_horas = df_horas.append(pd.DataFrame(GeoVictoria.GetAttendance(person=','.join(cod_fun[(contador * 48) + 1:(contador * 48) + 48]))))
    contador = contador + 1

df_horas = df_horas.reset_index(drop=True)

In [95]:
df_ponto = pd.DataFrame(columns=['Cod. Funcionario',
                                 'Nome completo',
                                 'Turno',
                                 'Permissao',
                                 'Data',
                                 'H. Ent1',
                                 'H. Sai1',
                                 'Intervalo',
                                 'H. Ent2',
                                 'H. Sai2'])

df_ponto['Cod. Funcionario'] = df_horas['identifier']
df_ponto['Data'] = df_horas['begins']
df_ponto['H. Ent1'] = df_horas['first_punch']
df_ponto['H. Sai1'] = df_horas['second_punch']
df_ponto['H. Ent2'] = df_horas['third_punch']
df_ponto['H. Sai2'] = df_horas['fourth_punch']

df_ponto['Data'] = df_ponto['Data'].apply(lambda x: dt.date(int(x[:4]), int(x[4:6]), int(x[6:8])) if len(x) > 10 else None)

df_ponto['H. Ent1'] = df_ponto['H. Ent1'].apply(lambda x: f'{x[8:10]}:{x[10:12]}' if len(x) > 10 else '--:--')
df_ponto['H. Sai1'] = df_ponto['H. Sai1'].apply(lambda x: f'{x[8:10]}:{x[10:12]}' if len(x) > 10 else '--:--')
df_ponto['H. Ent2'] = df_ponto['H. Ent2'].apply(lambda x: f'{x[8:10]}:{x[10:12]}' if len(x) > 10 else '--:--')
df_ponto['H. Sai2'] = df_ponto['H. Sai2'].apply(lambda x: f'{x[8:10]}:{x[10:12]}' if len(x) > 10 else '--:--')

df_ponto['Intervalo'] = df_ponto['H. Ent2'].apply(lambda x: ((int(x[:2]) * 60) + (int(x[3:5]))) if '--:--' != x else 0) - df_ponto['H. Sai1'].apply(lambda x: ((int(x[:2]) * 60) + (int(x[3:5]))) if '--:--' != x else 0)
df_ponto['Intervalo'] = df_ponto['Intervalo'].apply(lambda x: FolhaPonto.minuto_hora(x)).replace('00:00', '--:--')

for i in df_ponto.index:
    t_trabralhado = FolhaPonto.minuto_hora((FolhaPonto.hora_minuto(df_ponto.loc[i, 'H. Sai1']) - FolhaPonto.hora_minuto(df_ponto.loc[i, 'H. Ent1'])) + (FolhaPonto.hora_minuto(df_ponto.loc[i, 'H. Sai2']) - FolhaPonto.hora_minuto(df_ponto.loc[i, 'H. Ent2']))).replace('00:00', '--:--')

    if (df_ponto.loc[i, 'H. Ent1'] != '--:--' and df_ponto.loc[i, 'H. Sai1'] != '--:--') and df_ponto.loc[i, 'H. Ent2'] == '--:--':
        df_ponto.loc[i, 'Situação'] = 'Esqueceu'
    elif df_ponto.loc[i, 'H. Ent1'] != '--:--' and df_ponto.loc[i, 'H. Sai1'] != '--:--':
        df_ponto.loc[i, 'Situação'] = 'Almoçou'
    elif df_ponto.loc[i, 'H. Ent1'] != '--:--' and df_ponto.loc[i, 'H. Sai2'] != '--:--':
        df_ponto.loc[i, 'Situação'] = 'Saiu direto'
    else:
        df_ponto.loc[i, 'Situação'] = '--:--'
        
    if df_ponto.loc[i, 'Situação'] == 'Esqueceu':
        df_ponto.loc[i, 'Trabalhado'] = '--:--'
    else:
        df_ponto.loc[i, 'Trabalhado'] = t_trabralhado



    weekend_day = df_ponto.loc[i, 'Data'].strftime('%w')
    try:
        df_ponto.loc[i, 'Nome completo'] = df_funcionários[df_funcionários['Identifier'] == df_ponto.loc[i, 'Cod. Funcionario']]['Name'].values[0] + ' ' + df_funcionários[df_funcionários['Identifier'] == df_ponto.loc[i, 'Cod. Funcionario']]['LastName'].values[0]
        
        if (weekend_day != '0') & (weekend_day != '6'):
            df_ponto.loc[i, 'Turno'] = df_funcionários[df_funcionários['Identifier'] == df_ponto.loc[i, 'Cod. Funcionario']]['weeklyHoursCode'].values[0]
        else:
            df_ponto.loc[i, 'Turno'] = 'Nenhum'
            
        for (id, permis) in lista_tp_permis:
            if id == df_permis[(df_permis['Identifier'] == df_ponto.loc[i, 'Cod. Funcionario']) & (df_permis['PERMISSION_START'] == df_ponto.loc[i, 'Data'].strftime('%Y%m%d000000'))]['ID_PERMISSION_TYPE'].values[0]:
                if permis == None:
                    df_ponto.loc[i, 'Permissao'] = 'Nenhum'
                else:
                    df_ponto.loc[i, 'Permissao'] = permis

    except:
        df_ponto.loc[i, 'Permissao'] = 'Nenhum'
        continue

In [96]:
df_ponto['Turno'].unique()

array(['44 horas semanais', 'Nenhum', None, nan, '35 horas semanais',
       '30 horas semanais '], dtype=object)

In [97]:
for i in df_ponto.index:
    if (df_ponto.loc[i, 'Permissao'] == 'Nenhum') & (df_ponto.loc[i, 'Turno'] == '44 horas semanais'):
        cumprimento = '08:48'
    elif (df_ponto.loc[i, 'Permissao'] == 'Nenhum') & (df_ponto.loc[i, 'Turno'] == '35 horas semanais'):
        cumprimento = '07:00'
    elif (df_ponto.loc[i, 'Permissao'] == 'Nenhum') & (df_ponto.loc[i, 'Turno'] == '30 horas semanais '):
        cumprimento = '06:00'
    else:
        cumprimento = '--:--'

    df_ponto.loc[i, 'H. Extra / Faltante'] = FolhaPonto.minuto_hora(FolhaPonto.hora_minuto(df_ponto.loc[i, 'Trabalhado']) - FolhaPonto.hora_minuto(cumprimento)).replace('00:00','--:--')
    cumprimento = '--:--'


df_ponto[(df_ponto['Cod. Funcionario'] == '003609')]# | (df_ponto['Cod. Funcionario'] == '004308')]

,Cod. Funcionario,Nome completo,Turno,Permissao,Data,H. Ent1,H. Sai1,Intervalo,H. Ent2,H. Sai2,Situação,Trabalhado,H. Extra / Faltante
4906,003609,Fernanda Couto costa,44 horas semanais,Nenhum,2022-11-21,09:13,12:47,00:32,13:19,19:03,Almoçou,09:18,00:30
4907,003609,Fernanda Couto costa,44 horas semanais,Nenhum,2022-11-22,09:38,12:41,00:40,13:21,19:13,Almoçou,08:55,00:07
4908,003609,Fernanda Couto costa,44 horas semanais,Nenhum,2022-11-23,11:20,12:52,00:38,13:30,19:35,Almoçou,07:37,-01:11
4909,003609,Fernanda Couto costa,44 horas semanais,Nenhum,2022-11-24,09:09,--:--,--:--,--:--,14:02,Saiu direto,04:53,-03:55
4910,003609,Fernanda Couto costa,44 horas semanais,Nenhum,2022-11-25,09:10,12:18,00:44,13:02,19:01,Almoçou,09:07,00:19
4911,003609,Fernanda Couto costa,Nenhum,Nenhum,2022-11-26,--:--,--:--,--:--,--:--,--:--,--:--,--:--,--:--
4912,003609,Fernanda Couto costa,Nenhum,Nenhum,2022-11-27,--:--,--:--,--:--,--:--,--:--,--:--,--:--,--:--
4913,003609,Fernanda Couto costa,44 horas semanais,Nenhum,2022-11-28,09:15,12:12,02:45,14:57,19:14,Almoçou,07:14,-01:34
4914,003609,Fernanda Couto costa,44 horas semanais,Nenhum,2022-11-29,09:33,12:25,00:35,13:00,19:27,Almoçou,09:19,00:31
4915,003609,Fernanda Couto costa,44 horas semanais,Nenhum,2022-11-30,09:29,12:16,00:52,13:08,21:33,Almoçou,11:12,02:24


In [98]:
FolhaPonto.calculo(df_ponto[(df_ponto['Cod. Funcionario'] == '003609')]['H. Extra / Faltante'])

'-10:56'

In [99]:
df_ponto[df_ponto['Turno'] == '35 horas semanais']

,Cod. Funcionario,Nome completo,Turno,Permissao,Data,H. Ent1,H. Sai1,Intervalo,H. Ent2,H. Sai2,Situação,Trabalhado,H. Extra / Faltante
4875,003608,Gustavo Santos oliveira,35 horas semanais,Nenhum,2022-11-21,07:59,12:19,01:00,13:19,15:35,Almoçou,06:36,-00:24
4876,003608,Gustavo Santos oliveira,35 horas semanais,Atestado Médico,2022-11-22,--:--,--:--,--:--,--:--,--:--,--:--,--:--,--:--
4877,003608,Gustavo Santos oliveira,35 horas semanais,Atestado Médico,2022-11-23,--:--,--:--,--:--,--:--,--:--,--:--,--:--,--:--
4878,003608,Gustavo Santos oliveira,35 horas semanais,Atestado Médico,2022-11-24,--:--,--:--,--:--,--:--,--:--,--:--,--:--,--:--
4879,003608,Gustavo Santos oliveira,35 horas semanais,Atestado Médico,2022-11-25,--:--,--:--,--:--,--:--,--:--,--:--,--:--,--:--
4882,003608,Gustavo Santos oliveira,35 horas semanais,Nenhum,2022-11-28,08:00,--:--,--:--,--:--,12:14,Saiu direto,04:14,-02:46
4883,003608,Gustavo Santos oliveira,35 horas semanais,Nenhum,2022-11-29,07:58,11:55,01:11,13:06,16:28,Almoçou,07:19,00:19
4884,003608,Gustavo Santos oliveira,35 horas semanais,Nenhum,2022-11-30,07:59,12:00,01:12,13:12,16:30,Almoçou,07:19,00:19
4885,003608,Gustavo Santos oliveira,35 horas semanais,Nenhum,2022-12-01,07:56,12:56,01:01,13:57,18:00,Almoçou,09:03,02:03
4886,003608,Gustavo Santos oliveira,35 horas semanais,Nenhum,2022-12-02,07:57,--:--,--:--,--:--,14:09,Saiu direto,06:12,-00:48


In [100]:
# for i in df_ponto.index:
#     df_ponto.loc[i, 'Data'] = df_ponto.loc[i, 'Data'].strftime('%d/%m/%Y, %A')

#     for (english, portuguese) in dias_semana:
#         if english in df_ponto.loc[i, 'Data']:
#             df_ponto.loc[i, 'Data'] = df_ponto.loc[i, 'Data'].replace(english, portuguese)

In [101]:
df_funcionários.loc[df_funcionários[df_funcionários['Identifier'] == '003791'].index[0]]

Name                                       Janderson
LastName                              Da silva souza
Identifier                                    003791
Email                    jandersonfluzao12@gmail.com
Adress                                          None
Phone                                           None
Enabled                                            1
Custom1                                         None
Custom2                                         None
Custom3                                         None
GroupIdentifier                                 1029
GroupDescription                     Logística - SOF
ContractDate                                        
UserProfile                                     User
userScheduler                 44hrs 72almoço Correto
userCompanyIdentifier                38063210/000100
weeklyHoursCode                    44 horas semanais
endContractDate                                     
positionIdentifier            a8NoYYV-agZA64an

In [102]:
len(GeoVictoria.GetAttendance(person='002934,002734'))

62